# PGEN(Prompt-based domain text Generation)
사전 학습된 언어 모델(GPT-2 등)을 활용하여 도메인 내 텍스트 데이터를 대규모로 생성하는 방법
- [고려대학교 구름](https://github.com/nlpai-lab/KULLM)

## 0. 라이브러리 및 데이터 불러오기

In [1]:
import numpy as np
import pandas as pd
#from torch.utils.data import Dataset, DataLoader
#import torch
#import torch.optim as optim

## 1. 데이터 불러오기

In [6]:
df = pd.read_csv('/content/drive/MyDrive/0000/project_coda/dataset/NIASL2020_LLM.csv')
df = df[df.g_type=='SEN']
df = df[['start_end_time', 'gloss']]
df['gloss'] = df['gloss'].astype(str).apply(lambda x: x.replace('|', ' '))
df.text = ''
df

,start_end_time,gloss
48000,"[[1.975, 2.856], [2.989, 3.489], [3.608, 4.203...",기차 곳 내리다 맞다
48001,"[[1.626, 2.389], [2.477, 2.977], [3.09, 3.59],...",편지 저기 곳 다음
48002,"[[2.362, 3.974]]",위험
48003,"[[1.856, 2.638], [2.799, 3.388], [3.557, 4.186...",스타벅스 곳 저기 전 도착 차내리다
48004,"[[1.213, 1.899], [2.107, 2.607], [2.843, 3.343...",여기 송파 지하철 맞다
...,...,...
79995,"[[2.028, 2.528], [2.647, 3.483]]",그남자 맞다
79996,"[[1.959, 2.668], [2.782, 3.282], [3.42, 4.676]]",손바닥찍다 가다 내리다
79997,"[[1.847, 2.096], [2.41, 2.774], [3.095, 4.134]]",핸드폰 잃어버리다 찾다
79998,"[[1.407, 2.179], [2.37, 3.278]]",가방 실종


## 2. LLM 불러오기

In [9]:
!pip install torch transformers==4.38.2 accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 2.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_6

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer

MODEL_DIR = "nlpai-lab/KULLM3"
model = AutoModelForCausalLM.from_pretrained(MODEL_DIR, torch_dtype=torch.float16).to("cuda")
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=1)
prompter = Prompter("kullm")

In [ ]:
def infer(instruction="", input_text=""):
    prompt = prompter.generate_prompt(instruction, input_text)
    output = pipe(
        prompt, max_length=512,
        temperature=0.2,
        repetition_penalty=3.0,
        num_beams=5,
        eos_token_id=2
    )
    s = output[0]["generated_text"]
    result = prompter.get_response(s)

    return result

In [ ]:
instruction = """
백종원은 1966년 충남 예산군에서 집안의 종손으로 태어났다.
중학교 시절 상경해 강남 8학군 서울고등학교[23]를 졸업하고 연세대학교에 입학했다.
고등학교 졸업 직후엔 잠시 서울특별시 장한평에 위치한 중고차 시장에서 자동차 중개업자로 활동하기도 했다.
2019년, KBS의 토크쇼 프로그램인 대화의 희열에 출연하여 어렸을 적 이야기를 많이 했는데,
만석꾼이었던 증조할아버지 백영기(白榮基)의 피를 이어받은 영향인지
어렸을 때부터 장사꾼 기질이 있다고 스스로 자각하고 있었다고 한다.
9살 때에는 산에 놀러갔다가 본 버섯 농장에서 별다른 투자도 안 한 거 같은데
돈이 된다는 이야기를 듣고 꿈을 버섯 농사로 정한 적도 있었고,
초등학교 4학년때는 캔이 아닌 병에 음료가 나올 시절에 음료수 병을 보고 '저게 돈이 될 것 같다'
고 생각해 학교 리어카를 빌려 오락 시간과 보물찾기 같은 시간을 다 건너뛰고
리어카 6개 분량의 공병을 모아서 고물상에 갖다 팔아 큰 돈을 벌었다고 한다.
그리고 5학년 1학기까지 이렇게 돈을 벌었고, 방위성금으로 다 냈다고 한다.
""" # [출처: 나무위키]

result = infer(instruction=instruction, input_text="백종원의 증조할아버지는 누구?")
print(result)